## Spark S3 proxy secret
### This notebook demonstrates an example of creation of s3 secret for spark application consumption.

In [ ]:
import os
import subprocess


def deploy_s3_secret(namespace, spark_secret):
    try:
        # Run kubectl apply command using subprocess
        subprocess.run(['kubectl', 'delete', 'secret', spark_secret, '-n', namespace], check=False)
        subprocess.run(['kubectl', 'create', 'secret', 'generic', spark_secret, '-n', namespace , '--from-file=spark-defaults.conf'], check=True)
        print("Secret creation successful!")
    except subprocess.CalledProcessError as e:
        print(f"Secret creation  failed. Error: {e}")


s3_access_data = "spark.hadoop.fs.s3a.access.key EXAMPLE_ACCESS_KEY"
s3_secret_data = "spark.hadoop.fs.s3a.secret.key EXAMPLE_SECRET_KEY"

s3_data = s3_access_data.replace('EXAMPLE_ACCESS_KEY', os.environ['AUTH_TOKEN'])
s3_data += "\n" + s3_secret_data.replace("EXAMPLE_SECRET_KEY", "s3")

namespace = os.environ['USER']
spark_secret = "spark-s3-secret"

#Save data to a file spark-defaults.conf
with open('spark-defaults.conf', 'w') as file:
    file.write(s3_data)


# Call the function to deploy the Kubernetes secret
deploy_s3_secret(namespace, spark_secret)